In [33]:
from amplpy import AMPL
import pandas as pd

In [34]:
rideshares = pd.read_csv('./rideshare_tg22.csv')
airports = ['LAX', 'ONT', 'SNA', 'BUR']
car_types = ['UberX', 'UberXL']

In [35]:
ez_rideshares = rideshares#[rideshares['Departure Date'] == '5/11/24']
ez_rideshares.rename(columns={'Departure Time (Pacific Daylight Time)': 'Flight depart'}, inplace=True)

In [36]:
cost_data = {
    ('LAX', 'UberX'): 90, ('LAX', 'UberXL'): 120,
    ('ONT', 'UberX'): 30, ('ONT', 'UberXL'): 40,
    ('SNA', 'UberX'): 90, ('SNA', 'UberXL'): 120,
    ('BUR', 'UberX'): 90, ('BUR', 'UberXL'): 120
}

In [37]:
capacity_data = {
    'UberX': 4,
    'UberXL': 6
}

In [38]:
travel_time_data = {
    'LAX': 1.5,
    'ONT': 0.5,
    'SNA': 1.5,
    'BUR': 1.5
}

In [39]:
pricing_multiplier = [1.00, 1.02, 1.02, 1.03, 1.03, 0.99, 1.04, 1.03, 1.03, 1.08, 1.08, 1.14, 1.14, 1.19, 1.19, 1.20, 1.20, 1.30, 1.14, 1.10, 1.13, 1.12, 1.15, 1.17, 1.16, 1.17, 1.17, 1.20, 1.20, 1.19, 1.23, 1.22, 1.26, 1.24, 1.27, 1.25, 1.25, 1.18, 1.18, 1.19, 1.19, 1.17, 1.08, 1.12, 1.12, 1.07, 1.07, 1.00 ]

In [40]:
ez_rideshares['Flight depart'] = pd.to_datetime(ez_rideshares['Flight depart'], format='%H:%M')
ez_rideshares['Flight times'] = ez_rideshares['Flight depart'].dt.hour + 0.5 * (ez_rideshares['Flight depart'].dt.minute//30)

In [41]:
ampl = AMPL()
ampl.eval(r"""
    set A;
    set S;
    set P = 0.5..24 by 0.5;  # time of arrival to airport
    set C;

    param cost{A, C};
    param capacity{C};
    param travelTime{A};
    param flightTime{S, A};

    var numCars{A, P, C} >= 0 integer;  # x
    var leaving{A, S, P} binary;  # 1 if student s leaves for airport a at time p, else 0
    var departureTime{A, S};

    minimize objective: sum{a in A, p in P, c in C} numCars[a, p, c]*cost[a, c];

    subject to notTooLateConstraint{a in A, s in S, p in P}:
        p - leaving[a, s, p]*(flightTime[s, a] - 2 - travelTime[a]) <= 100*(1 - leaving[a, s, p]);

    subject to notTooEarlyConstraint{a in A, s in S, p in P}:
        leaving[a, s, p]*(flightTime[s, a] - 4 - travelTime[a]) - p <= 100*(1 - leaving[a, s, p]);

    subject to capacityConstraint{a in A, p in P}:
        sum{s in S} leaving[a, s, p] <= sum{c in C} numCars[a, p, c] * capacity[c];

    subject to onlyOneActiveY{s in S}:
        sum{a in A, p in P} leaving[a, s, p] = 1;

    subject to defineDepartureTime{a in A, s in S}:
        departureTime[a, s] = sum{p in P} leaving[a, s, p] * p;

""")

In [42]:
# reset index on ez_rideshares to start at 0
ez_rideshares.reset_index(drop=True, inplace=True)

In [43]:
# set flight time data equal to Flight Times form ez_rideshares
# format: {(ez_rideshares['Name'][i], 'ONT') : ez_rideshares['Flight times'][i]}

raw_flight_time_data = {}
for i in range(len(ez_rideshares)):
    raw_flight_time_data[(ez_rideshares['Name'][i], 'ONT')] = ez_rideshares['Flight times'][i]

In [44]:
flight_time_data = {}
for s in ez_rideshares['Name']:
    for j in airports:
        if (s, j) in raw_flight_time_data:
            flight_time_data[(s, j)] = raw_flight_time_data[(s, j)]
        else:
            flight_time_data[(s, j)] = 0

In [45]:
# read ez_rideshares into ampl2
ampl.set['A'] = airports
ampl.set['S'] = ez_rideshares['Name']
ampl.set['C'] = car_types

ampl.getParameter("cost").setValues(cost_data)
ampl.getParameter("capacity").setValues(capacity_data)
ampl.getParameter("travelTime").setValues(travel_time_data)
ampl.getParameter("flightTime").setValues(flight_time_data)


In [46]:
# run the model
ampl.solve(solver='gurobi')

Gurobi 11.0.3:Gurobi 11.0.3: optimal solution; objective 200
119 simplex iterations
1 branching node


In [47]:
# display the results
student_airport_times = {}
for student in ez_rideshares['Name']:
    for a in airports:
        x = ampl.getValue(f'departureTime["{a}", "{student}"]')
        if x != 0:
            student_airport_times[student] = (a, x)
student_airport_times

{'Nithya Yeluri': ('ONT', 2.5),
 'Nina Jobanputra': ('ONT', 8.5),
 'Aanya Pratapneni ': ('ONT', 8.5),
 'Will Sedo': ('ONT', 8.5),
 'Marcella Todd': ('ONT', 11.5),
 'Kaanthi Pandhigunta': ('ONT', 11.5),
 'Cevi Bainton': ('ONT', 11.5),
 'Ally Dye': ('ONT', 11.5),
 'Maya Maranto': ('ONT', 14.5),
 'Alec Vercruysse': ('ONT', 14.5),
 'Rebecca Preis': ('ONT', 14.5),
 'Susan Li': ('ONT', 16.5),
 'Tanvi Krishnan ': ('ONT', 16.5),
 'Jennifer Li': ('ONT', 16.5),
 'Allison Marten': ('ONT', 16.5),
 'Aldrin Feliciano': ('ONT', 16.5),
 'Julianna Schalkwyk': ('ONT', 19),
 'Charles Weismann': ('ONT', 2.5),
 'Katrina Nguyen': ('ONT', 2.5),
 'Bennet Matazzoni': ('ONT', 11.5),
 'Thaxter Shaw': ('ONT', 19),
 'Alicia Krasner': ('ONT', 8.5),
 'Laura Vairus': ('ONT', 14.5),
 'Ashley Kim': ('ONT', 16.5)}

In [50]:
for a in airports:
    for i in range(1, 49):
        for car in car_types:
            numCars = ampl.getValue(f"numCars['{a}', {i/2}, '{car}']")
            if numCars > 0:
                print(f'Time {i/2} has {numCars} {car} car(s) to {a}')

Time 2.5 has 1 UberX car(s) to ONT
Time 8.5 has 1 UberX car(s) to ONT
Time 11.5 has 1 UberXL car(s) to ONT
Time 14.5 has 1 UberX car(s) to ONT
Time 16.5 has 1 UberXL car(s) to ONT
Time 19.0 has 1 UberX car(s) to ONT
